In [1]:
%run 'C:\dev\prod\metrikaHelper.py'
import pandas as pd
import psycopg2
import math
import numpy
from datetime import timedelta
from datetime import datetime
from pandas import ExcelWriter
def getVipProjectByDomain(domain):
    conn = psycopg2.connect("dbname='CRM' user='read_only' host='192.168.10.10' password='User_ro'")
    cur = conn.cursor()
    cur.execute("SELECT p.projectid, p.domain, p2.login, p2.password FROM project AS p INNER JOIN ProjectMetrikaData AS p2 ON p.UniqueId = p2.ProjectUniqueId WHERE domain=%s ORDER BY p.projectid", [domain])
    result = []
    rows = cur.fetchall()
    for row in rows:
        result.append(getVipProjectByRow(row))
    return result
def getVipProjectByRow(row):
    return {'projectid': row[0], 'domain': row[1], 'metrikalogin': row[2], 'metrikapassword': row[3]}
def getCPATargetsByProjectID(projectID):
    conn = psycopg2.connect("dbname='Cubo' user='read_only' host='192.168.10.8' password='User_ro'")
    cur = conn.cursor()
    cur.execute("SELECT targetstatus, targetid, targetname FROM metricstarget WHERE projectid=%s ORDER BY id DESC", [projectID])
    result = []
    rows = cur.fetchall()
    for row in rows:
        result.append(getCPATargetByRow(row))
    return result
def getCPATargetByRow(row):
    return {'targetstatus': row[0], 'targetid': row[1], 'targetname': row[2]}
project = u'simple-pro.com'
VipProjectInfo = getVipProjectByDomain(project)
print VipProjectInfo
metrikaCredentials = getMetrikaCredentialsByLPD({'yandexLogin':VipProjectInfo[0]['metrikalogin'],
                                                 'yandexPassword':VipProjectInfo[0]['metrikapassword'],
                                                 'domain':VipProjectInfo[0]['domain']})
print metrikaCredentials

[{'metrikalogin': 'cubo.client2@yandex.ru', 'projectid': 2371741L, 'domain': 'simple-pro.com', 'metrikapassword': 'idev2015'}]
{'Token': u'00687c8af3aa4b4bbddea008faae5bfa', 'counterID': [7971226]}


In [2]:
goalsMetrics = ','.join(['ym:s:goal'+str(target['targetid'])+'visits' for target in getCPATargetsByProjectID(VipProjectInfo[0]['projectid'])])

In [3]:
goalsMetrics

'ym:s:goal6578510visits,ym:s:goal5452244visits,ym:s:goal5452241visits'

Детализация по транзакциям

In [7]:
url = 'https://beta.api-metrika.yandex.ru/stat/v1/data.csv?dimensions=ym:s:UTMSource,ym:s:UTMMedium,ym:s:UTMCampaign,ym:s:UTMTerm,ym:s:UTMContent,ym:s:purchaseID&metrics=ym:s:visits,ym:s:ecommerceRevenue,ym:s:goal6578510visits,ym:s:goal5452244visits,ym:s:goal5452241visits,ym:s:ecommercePurchases&date1=2016-01-25&date2=2016-04-07&ids=7971226&oauth_token=00687c8af3aa4b4bbddea008faae5bfa&limit=100000&offset=1'
transactions_df = pd.read_csv(url,encoding='utf-8')

Статистика по доходу и количеству покупок

In [13]:
url = 'https://beta.api-metrika.yandex.ru/stat/v1/data.csv?dimensions=ym:s:UTMSource,ym:s:UTMMedium,ym:s:UTMCampaign,ym:s:UTMTerm,ym:s:UTMContent&metrics=ym:s:visits,ym:s:ecommerceRevenue,ym:s:goal6578510visits,ym:s:goal5452244visits,ym:s:goal5452241visits,ym:s:ecommercePurchases&date1=2016-01-25&date2=2016-04-07&ids=7971226&oauth_token=00687c8af3aa4b4bbddea008faae5bfa&limit=100000&offset=1'
summary_stat = pd.read_csv(url,encoding='utf-8')

In [15]:
transactions_df

,"﻿""UTM Source""",UTM Medium,UTM Campaign,UTM Term,UTM Content,ID покупки,Визиты,Доход,Целевые визиты,Целевые визиты.1,Целевые визиты.2,Количество покупок
0,Итого и средние,NaN,NaN,NaN,NaN,NaN,10321,660255,9,10,12,9
1,cubo,adwords,shkafy,Не определено,Не определено,327,1,11820,1,0,0,1
2,cubo,direct,Не определено,Не определено,none.0.no,333,1,362500,1,0,0,1
3,cubo,direct,4290818289,Шкаф для реактивов,premium.3.no,322,1,34310,1,0,0,1
4,cubo,direct,4290818395,ооо спецбалтмебель,premium.1.no,343,1,18540,1,0,0,1
5,cubo,direct,4290820705,Тумба медицинская с мойкой,none.0.no,344,1,15950,1,0,0,1
6,cubo,direct,4383648238,лабораторная мебель,none.0.no,318,1,13660,1,0,0,1
7,cubo,direct,4384501176,вытяжной шкаф,none.0.no,345,1,35060,1,0,0,1
8,cubo,direct,4941715850,спецбалтмебель официальный сайт,premium.1.no,342,1,161725,1,0,0,1
9,cubo,direct,{phrase_id},{keyword},{position_type}.{position}.{addphrases},317,1,6690,1,0,0,1
